# Assignment 2: Generative Adversarial Network (GAN) for Image Generation
**Student Name:** Lakshit Gupta  
**Enrollment No:** E23CSEU0992  
**Date:** January 20, 2026

## Objective
To implement and train a basic GAN model to generate new synthetic images and evaluate the quality of generated samples over training epochs.

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

print("TensorFlow:", tf.__version__)


#GPU allow growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


TensorFlow: 2.19.0


In [13]:
#config
dataset_choice = input("Enter dataset (mnist/fashion) : ").strip().lower() or "mnist"
epochs = int(input("Enter number of epochs : ").strip() or 50)
batch_size = int(input("Enter batch size : ").strip() or 128)
noise_dim = int(input("Enter noise dimensions : ").strip() or 100)
learning_rate = float(input("Enter learning rate : ").strip() or 0.0002)
save_interval = int(input("Enter save interval : ").strip() or 5)
# Fix seeds (reproducibility-ish)
tf.random.set_seed(42)
np.random.seed(42)


Enter dataset (mnist/fashion) : fashion
Enter number of epochs : 50
Enter batch size : 256
Enter noise dimensions : 100
Enter learning rate : 0.0009
Enter save interval : 5


In [14]:
if dataset_choice == 'mnist':
    (x_train, y_train), _ = mnist.load_data()
    num_classes = 10
else:
    (x_train, y_train), _ = fashion_mnist.load_data()
    num_classes = 10

# Normalize images to [-1, 1]
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

image_shape = x_train.shape[1:]



29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [15]:
def build_generator():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(noise_dim,)),
        layers.Dense(256),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),

        layers.Dense(512),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),

        layers.Dense(1024),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),

        layers.Dense(np.prod(image_shape), activation='tanh'),
        layers.Reshape(image_shape)
    ])
    return model


In [16]:
def build_discriminator():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=image_shape),
        layers.Flatten(),
        layers.Dense(512),
        layers.LeakyReLU(0.2),

        layers.Dense(256),
        layers.LeakyReLU(0.2),

        layers.Dense(1, activation='sigmoid')
    ])
    return model


In [17]:
optimizer = Adam(learning_rate, 0.5)

discriminator = build_discriminator()
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

generator = build_generator()

z = layers.Input(shape=(noise_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)

gan = tf.keras.Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)


In [18]:
os.makedirs("generated_samples", exist_ok=True)

def save_images(epoch):
    noise = np.random.normal(0, 1, (25, noise_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5  # scale to [0,1]

    fig, axs = plt.subplots(5, 5, figsize=(5,5))
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i,j].imshow(gen_imgs[idx, :, :, 0], cmap='gray')
            axs[i,j].axis('off')
            idx += 1
    plt.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()


In [19]:
for epoch in range(1, epochs + 1):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    fake_imgs = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    print(f"Epoch {epoch}/{epochs} | D_loss: {d_loss[0]:.2f} | "
          f"D_acc: {d_loss[1]*100:.2f}% | G_loss: {g_loss:.2f}")

    if epoch % save_interval == 0:
        save_images(epoch)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50 | D_loss: 0.71 | D_acc: 38.18% | G_loss: 0.56
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 2/50 | D_loss: 0.73 | D_acc: 32.16% | G_loss: 0.47
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 3/50 | D_loss: 0.73 | D_acc: 31.35% | G_loss: 0.40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 4/50 | D_loss: 0.74 | D_acc: 30.42% | G_loss: 0.34
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 5/50 | D_loss: 0.74 | D_acc: 29.44% | G_loss: 0.29
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 6/50 | D_loss: 0.75 | D_acc: 28.76% | G_loss: 0.26
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 7/50 | D_loss: 0.76 | D_acc: 28.54% | G_loss: 0.23
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 8/50 | D_loss: 0.77 | D_acc: 27.98% | G_loss: 0.21
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 9/50 | D_loss: 0.78 | D_acc: 27.52% | G_loss: 0.19
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 10/50 | D_loss: 0.80 | D_acc: 27.22% | G_loss: 0.17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
8/8 ━━━━━━

In [20]:
os.makedirs("final_generated_images", exist_ok=True)

noise = np.random.normal(0, 1, (100, noise_dim))
generated_images = generator.predict(noise)
generated_images = 0.5 * generated_images + 0.5

for i in range(100):
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')
    plt.savefig(f"final_generated_images/img_{i}.png")
    plt.close()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [23]:
classifier = tf.keras.Sequential([
    tf.keras.Input(shape=image_shape),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


classifier.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

classifier.fit(x_train, y_train, epochs=3, batch_size=128, verbose=0)


In [22]:
predictions = classifier.predict(generated_images)
predicted_labels = np.argmax(predictions, axis=1)

unique, counts = np.unique(predicted_labels, return_counts=True)
label_distribution = dict(zip(unique, counts))

print("Label Distribution of Generated Images:")
print(label_distribution)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
Label Distribution of Generated Images:
{np.int64(6): np.int64(6), np.int64(8): np.int64(94)}
